In [1]:
import arcpy

# Full paths to the layers
input_layer = r"C:\Users\Sea Grant\Documents\GitHub\WW_Overlay_2024\Data\Outputs\TaxPlat_No_P1\TaxPlat_No_P1.shp"
osds_layer = r"C:\Users\Sea Grant\Documents\GitHub\WW_Overlay_2024\Data\OSDS\OahuOSDS\On-site_Sewage_Disposal_Systems_-_Oahu.shp"
footprint_layer = r"C:\Users\Sea Grant\Documents\GitHub\WW_Overlay_2024\Data\Building_Footprint_Centerpoints\Building_Footprint_Centerpoints.shp"
mains_layer = r"C:\Users\Sea Grant\Documents\GitHub\WW_Overlay_2024\Data\SewerMains\OahuSewerMains\Sewer_Main.shp"

# List of fields to add
fields_to_add = ["OSDS", "Footprint", "Mains"]

# Add fields to the layer
for field in fields_to_add:
    try:
        arcpy.AddField_management(input_layer, field, "TEXT")  # Add text field
        print(f"Successfully added field: {field}")
    except Exception as e:
        print(f"Failed to add field {field}: {e}")

# List of layers corresponding to each field
intersect_layers = {
    "OSDS": osds_layer,
    "Footprint": footprint_layer,
    "Mains": mains_layer
}

# Update cursor to update the fields based on intersection
with arcpy.da.UpdateCursor(input_layer, ["OID@", "SHAPE@"] + fields_to_add) as cursor:
    for row in cursor:
        oid = row[0]
        taxplat_geometry = row[1]

        # Check for intersections with each layer
        for i, (field, layer_path) in enumerate(intersect_layers.items(), start=2):  # Start=2 to skip OID and SHAPE
            intersects = False

            # Check if the TaxPlat geometry intersects with any feature in the intersecting layer
            with arcpy.da.SearchCursor(layer_path, ["SHAPE@"]) as intersect_cursor:
                for intersect_row in intersect_cursor:
                    intersect_geometry = intersect_row[0]
                    if taxplat_geometry.overlaps(intersect_geometry) or taxplat_geometry.intersects(intersect_geometry):
                        intersects = True
                        break

            row[i] = "yes" if intersects else "no"

        cursor.updateRow(row)

print("Field update process complete.")

Successfully added field: OSDS
Successfully added field: Footprint
Successfully added field: Mains


AttributeError: 'Polygon' object has no attribute 'intersects'

In [2]:
print(f"Geometry type: {type(taxplat_geometry)}")
print(f"Geometry type: {type(intersect_geometry)}")

Geometry type: <class 'arcpy.arcobjects.geometries.Polygon'>
Geometry type: <class 'arcpy.arcobjects.geometries.PointGeometry'>
